In [2]:
from thermoextrap import *
import xarray as xr
# ideal gas extrapolation
import thermoextrap.xtrapy.xpan_vol_ig as xpan_vol_ig
# "general extrapolation"
import thermoextrap.xtrapy.xpan_vol as xpan_vol

In [3]:
# test Data
# samples
n = int(1e5)
# number of x values
nv = 5
_u_offset = 0.0
_x_offset = 0.0
np.random.seed(0)
u = np.random.rand(n) + _u_offset
x = np.random.rand(n, nv) + _x_offset

In [4]:
#With original extrapolation package
class VolumeExtrapModelIG(ExtrapModel):
    """Class to hold information about an extrapolation in size for a 1D system (e.g. our ideal gas model)
    """
    #Can't go to higher order in practice, so don't return any symbolic derivatives
    #Instead, just use this to check and make sure not asking for order above 1
    def calcDerivFuncs(self):
        if self.maxOrder > 1:
            print('Volume extrapolation cannot go above 1st order without derivatives of forces.')
            print('Setting order to 1st order.')
            self.maxOrder = 1
        return None
    #And given data, calculate numerical values of derivatives up to maximum order
    #Will be very helpful when generalize to different extrapolation techniques
    #(and interpolation)
    def calcDerivVals(self, refL, x, W):
        """Calculates specific derivative values at B with data x and U up to max order.
        Returns these derivatives. Only go to first order for volume extrapolation. And
        here W represents the virial instead of the potential energy.
        """
        if x.shape[0] != W.shape[0]:
            print('First observable dimension (%i) and size of potential energy array (%i) don\'t match!'%(x.shape[0], W.shape[0]))
            return
        wT = np.array([W]).T
        avgX = np.average(x, axis=0)
        avgW = np.average(W)
        avgXW = np.average(x*wT, axis=0)
        derivVals = np.zeros((2, x.shape[1]))
        derivVals[0] = avgX
        derivVals[1] = (avgXW - avgX*avgW) / refL
        #Add the unique correction for the observable <x> in the ideal gas system
        #It turns out this is just <x> itself divided by L
        derivVals[1] += avgX / refL
        return derivVals

In [6]:
em = VolumeExtrapModelIG(maxOrder=1, refB=2.0, xData=x, uData=u)
xem = xpan_vol_ig.factory_extrapmodel(order=1, alpha0=2.0, uv=u, xv=x)

In [7]:
volumes = [0.1, 0.5, 1.5, 2.0]

In [8]:
np.testing.assert_allclose(em.predict(volumes), xem.predict(volumes))

In [9]:
from importlib import reload
reload(xpan_vol)

<module 'thermoextrap.xtrapy.xpan_vol' from '/Users/wpk/Documents/python/projects/thermodynamic-extrapolation/thermoextrap/xtrapy/xpan_vol.py'>

In [10]:
xem2 = xpan_vol.factory_extrapmodel(order=1, uv=u, xv=x, dxdqv=x, volume=2.0, ndim=1)

In [11]:
np.testing.assert_allclose(xem.predict(volumes), xem2.predict(volumes))

In [15]:
# resampling
ndat = x.shape[0]
nrep = 20
idx = np.random.choice(ndat, (nrep, ndat), True)

a = xem.resample(indices=idx)#.predict(volumes)
b = xem2.resample(indices=idx)#.predict(volumes)

np.testing.assert_allclose(a.predict(volumes),b.predict(volumes))

In [23]:
data = xpan_volume.DataCentralMomentsVals.from_vals(xv=x, uv=u, order=1, central=False)

In [24]:
x_extrap2 = xpan_volume.factory_extrapmodel(2.0, data)

In [27]:
np.testing.assert_allclose(x_extrap.predict([0.1, 0.5, 1.5, 2.0]), x_extrap2.predict([0.1, 0.5, 1.5, 2.0]))

In [16]:
from importlib import reload
reload(xpan_volume2)

<module 'thermoextrap.xtrapy.xpan_volume2' from '/Users/wpk/Documents/python/projects/thermodynamic-extrapolation/thermoextrap/xtrapy/xpan_volume2.py'>

In [ ]:
xpan_volume

In [46]:
x_extrap2 = xpan_volume2.factory_extrapmodel_volume(1, u, x, 2.0, 1.0, 1.0)

In [47]:
x_extrap2.predict(0.1)

<xarray.DataArray (val: 5)>
array([0.02489228, 0.0250577 , 0.02482549, 0.02495123, 0.0249524 ])
Coordinates:
    volume   float64 0.1
    dalpha   float64 -1.9
    volume0  float64 2.0
Dimensions without coordinates: val

In [39]:
x_extrap.data.xu

<xarray.DataArray 'x' (moment: 2, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612]])
Dimensions without coordinates: moment, val

In [31]:
x_extrap2.data.ndim

2.0

In [26]:
x_extrap.data.xu

<xarray.DataArray 'x' (moment: 2, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612]])
Dimensions without coordinates: moment, val

In [27]:
x_extrap.data.xu

<xarray.DataArray 'x' (moment: 2, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189],
       [0.25009013, 0.24913469, 0.25014455, 0.2493188 , 0.25043612]])
Dimensions without coordinates: moment, val

In [18]:
x_extrap2.predict([0.1, 0.5, 1.5, 2.0])

<xarray.DataArray (volume: 4, val: 5)>
array([[0.27516511, 0.27449868, 0.27511831, 0.27451691, 0.27558371],
       [0.37527424, 0.37427506, 0.37523544, 0.37434318, 0.37583623],
       [0.62554706, 0.62371604, 0.62552827, 0.62390886, 0.62646755],
       [0.75068347, 0.74843652, 0.75067468, 0.7486917 , 0.7517832 ]])
Coordinates:
  * volume   (volume) float64 0.1 0.5 1.5 2.0
    dalpha   (volume) float64 -0.9 -0.5 0.5 1.0
    volume0  float64 1.0
Dimensions without coordinates: val

In [20]:
np.testing.assert_allclose(orig_extrap.predict([0.1, 0.5, 1.5, 2.0]), x_extrap.predict([0.1, 0.5, 1.5, 2.0]))

In [22]:
#Now just check some features of the volume extrapolation
#Should throw value error if order above 1
x_extrap.predict([2.0], order=1)

<xarray.DataArray (volume: 1, val: 5)>
array([[0.50041065, 0.49899555, 0.50038186, 0.49912602, 0.50115189]])
Coordinates:
  * volume   (volume) float64 2.0
    dalpha   (volume) float64 0.0
    volume0  float64 2.0
Dimensions without coordinates: val